# 对APT模型的简单验证

In [1]:
import pandas as pd
import rqdatac as rq
import os

from dotenv import load_dotenv
load_dotenv()

rqdata_username = os.getenv('RQDATA_USERNAME')
rqdata_password = os.getenv('RQDATA_PASSWORD')
rq.init(rqdata_username, rqdata_password)

NameError: name 'os' is not defined

In [15]:
# 设置时间范围
start_date = '2020-01-01'
end_date = '2023-01-01'

# 获取资产和因子的价格数据
# 目标资产
asset_prices = rq.get_price('000651.XSHE', start_date=start_date, end_date=end_date, frequency='1d', fields='close')

# 随便选择三个因子：市场整体指数、10年国债利率(利率因子)，以及一个行业指数(行业因子)。
market_prices = rq.get_price('000001.XSHG', start_date=start_date, end_date=end_date, frequency='1d', fields='close')
industry_prices = rq.get_price('399006.XSHE', start_date=start_date, end_date=end_date, frequency='1d', fields='close')  # 例如使用创业板指数作为示例行业因子
interest_rates = rq.get_yield_curve(start_date, end_date, tenor='10Y')


In [16]:
# 计算收益率
# 需要注意：根据CAPM模型，标度预期收益率与市场投资组合预期收益率相关联，这是我们计算收益率的原因
# 但是interest_rates作为国债利率本身就是一个直接影响资产定价的因子，所以这里不需要再计算它的收益率

asset_returns = asset_prices.pct_change().dropna()
market_returns = market_prices.pct_change().dropna()
industry_returns = industry_prices.pct_change().dropna()

In [17]:
# 数据整理
asset_returns = asset_returns.reset_index()
asset_returns.drop(columns=['order_book_id'], inplace=True)

market_returns = market_returns.reset_index()
market_returns.drop(columns=['order_book_id'], inplace=True)

industry_returns = industry_returns.reset_index()
industry_returns.drop(columns=['order_book_id'], inplace=True)

interest_rates = interest_rates.reset_index()
interest_rates.rename(columns={'trading_date': 'date', '10Y': 'close'}, inplace=True)

In [18]:
asset_returns.set_index('date', inplace=True)
market_returns.set_index('date', inplace=True)
industry_returns.set_index('date', inplace=True)
interest_rates.set_index('date', inplace=True)

In [24]:
# 数据对齐
# data_aligned = pd.merge(asset_returns, market_returns, industry_returns, interest_rates, left_index=True, right_index=True, how='inner')
data_aligned = pd.concat([asset_returns, market_returns, industry_returns, interest_rates], axis=1, join='inner')
data_aligned.columns = ['asset_returns', 'market_returns', 'industry_returns', 'interest_rates']
data_aligned.head()

,asset_returns,market_returns,industry_returns,interest_rates
date,,,,
2020-01-03,-0.011782,-0.000458,0.001784,0.031428
2020-01-06,-0.018183,-0.000122,0.013022,0.031352
2020-01-07,0.021858,0.006938,0.017900,0.031379
2020-01-08,0.021984,-0.012210,-0.016118,0.031337
2020-01-09,0.015698,0.009126,0.027081,0.031184


In [25]:
import statsmodels.api as sm

# 增加常数项以进行回归
X = sm.add_constant(data_aligned[['market_returns', 'industry_returns', 'interest_rates']])
y = data_aligned['asset_returns']

# 执行线性回归
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          asset_returns   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     100.5
Date:                Tue, 15 Oct 2024   Prob (F-statistic):           2.11e-54
Time:                        13:12:12   Log-Likelihood:                 2010.7
No. Observations:                 727   AIC:                            -4013.
Df Residuals:                     723   BIC:                            -3995.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0012      0.008  

In [ ]:
# 在线性回归模型中，对于每个系数（包括常数项），我们进行的统计检验都是针对以下原假设（null hypothesis）：
# H_0： 每个解释变量对应的系数等于零
# 在这个前提下，p值代表着假设H0正确的概率，如果p值小于显著性水平（通常是0.05），我们就拒绝原假设，认为系数不等于零（也就相关）

# 如果p值小于预定的显著性水平，我们拒绝原假设，认为该变量的系数显著不为零。这表明该变量对被解释变量（如资产回报）有显著的统计影响
# 如果p值较大，未达到显著性水平，则不拒绝原假设，我们认为这个系数在统计上不显著，即该变量可能对被解释变量没有显著影响

# 可以发现误差项-也就是截距部分p值为0.888，这是一个非常高的值，也就是说对于const变量，H0的假设是正确的(系数等于0)
# 或者说在这个情况下的alpha是不存在的，因为它的系数没有显著的偏离0

# 如果说alpha显著的偏离0，则代表了某个可以通过套利而获得超额收益的机会；它也同时说明由于某些原因，市场对该资产出现错误定价